In [0]:
# В 2017 году Яндекс создал нейросеть-поэтессу по имени Зинаида Фолс. Стихи получились забавные, но работы предстоит еще много; надеюсь, в будущем работа редактора текста 
# с помощью таких продуктов станет легче. Давайте создадим подобную программу. В этом нам поможет рекуррентная нейронная сеть.
import tensorflow as tf

# numpy понадобится для работы с тензорами
import numpy as np

# os – для именования промежуточных файлов-чекпоинтов
import os

# а time – для инициализации эпох
import time

# Используем текст шекспировской "Трагедии о Кариолане"
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

In [20]:
# Откроем текст, вычитаем его и обозначим кодировку – формат преобразования Юникода, 8-бит
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# Отобразим количество знаков c помощью метода подстановки format(). len() определит длину объекта.
print ('Количество знаков: {}'.format(len(text)))

Количество знаков: 1115394


In [21]:
# Проверим, успешно ли прошло вычитывание. Используем срез и его синтаксиис – квадратные скобки и двоеточие. Так мы запрашиваем отображение символов от 0 до 250. 
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [22]:
# Создадим объект vocabulary, который отсортирует символы в алфавитном порядке, затем вычленит уникальные с помощью множества.
vocab = sorted(set(text))
# Отобразим количество уникальных символов 
print ('{} уникальных символов'.format(len(vocab)))

65 уникальных символов


In [23]:
# Прежде чем начать тренировку, нам необходимо создать численное представление строк. Создадим справочные таблицы, преобразующие символы char в индексы idx и обратно.
# Функция enumerate сгенерирует кортеж, состоящий из двух элементов - индекса и символа
char2idx = {u:i for i, u in enumerate(vocab)}
# Преобразуем словарь в массив
idx2char = np.array(vocab)

# Создадим объект "Текст как набор целочисленных значений"
text_as_int = np.array([char2idx[c] for c in text])
# Посмотрим, что получилось. В рехультирующем кортеже каждому встреченном символу присвоен уникальный ID. Фигурные скобки упростят восприятие.
print('{')
# Мы используем char – каждый символ кортежа char2idx, чтобы инициализировать цикл for. Переменная "underscore" (нижнее подчеркивание) в Python используется по общепринятому соглашению только в цикле
# в дальнейшем она будет отброшена. zip позволяет пройтись одновременно по нескольким итерируемым объектам – char2idx, range(20). Так мы запрашиваем 20 элементов кортежа.
for char,_ in zip(char2idx, range(20)):
    # "Двоеточие 4s" – это способ запросить срез с первого по четвертый элемент строки, обозначаемой s. Тоже самое для 'semicolon 3d', только символов меньше, 
    # и на выходе десятичное число.
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
# Весь кортеж отображать не будем, потому условно обозначим продолжение троеточием
print('  ...\n}')

{
  '\n':   0,
  ' ' :   1,
  '!' :   2,
  '$' :   3,
  '&' :   4,
  "'" :   5,
  ',' :   6,
  '-' :   7,
  '.' :   8,
  '3' :   9,
  ':' :  10,
  ';' :  11,
  '?' :  12,
  'A' :  13,
  'B' :  14,
  'C' :  15,
  'D' :  16,
  'E' :  17,
  'F' :  18,
  'G' :  19,
  ...
}


In [24]:
# Произведем тестовый мэппинг. Используем срез символов с нулевого по тринадцатый и тензор text_as_int
print ('{} ----> Определение соответствия символов индексам ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

'First Citizen' ----> Определение соответствия символов индексам ---- > [18 47 56 57 58  1 15 47 58 47 64 43 52]


In [25]:
# Поставим РНС задачу предсказать следующий символ для начала. Разделим текст на sequences длиной 100 символов.
seq_length = 100
# Подсчитаем, сколько таких последовательностей в одной эпохе тренировки. Для этого длину всего текста разделим на длину последовательности плюс единица.
examples_per_epoch = len(text)//(seq_length+1)

# Создадим тренировочный датасет символов, состоящий из слайсов.
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

# Где еще встретишь столько способов объявить цикл?! Мотаем на ус вместе. Методом take() в цикле for отобразим первые 5 элементов датасета. Здесь numpy копирует 
# символы в скаляр и выводит их.
for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

F
i
r
s
t


In [26]:
# Создадим объект sequences. Это результат конвертации char_dataset в т.н. партию (batch) – последовательность нужного размера. Отбросим остаток с помощью булевого 
# параметра drop_remainder.
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

# Посмотрим первые пять партий. Разрывы строк обозначаются как 'backslash n'. Метод join() снова объединяет символы в слова и предложения. 
for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


In [27]:
# Определим функцию "разделить вводные данные", определив аргумент-кусок (chunk) и отделим по первому и последнему символу у передаваемого функции объекта. 
# Это понадобится потом. 
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

# Мы подбираемся к тренировке еще ближе. Создадим датасет и применим к каждому элементу последовательности разделительную функцию.
dataset = sequences.map(split_input_target)

# Выведем примеры этих кусков, соединив символы
for input_example, target_example in dataset.take(1):
  print ('Вводные данные: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Целевые данные:', repr(''.join(idx2char[target_example.numpy()])))

Вводные данные:  'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Целевые данные: 'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


In [28]:
# Теперь мы пошагово отобразим, какая буква последует за какой. Используем векторы input_text и ее смещенного собрата target_text. Это пока имитация предсказания символа.
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Шаг {:4d}".format(i))
    print("  Ввод: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  Ожидаемый символ: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Шаг    0
  Ввод: 18 ('F')
  Ожидаемый символ: 47 ('i')
Шаг    1
  Ввод: 47 ('i')
  Ожидаемый символ: 56 ('r')
Шаг    2
  Ввод: 56 ('r')
  Ожидаемый символ: 57 ('s')
Шаг    3
  Ввод: 57 ('s')
  Ожидаемый символ: 58 ('t')
Шаг    4
  Ввод: 58 ('t')
  Ожидаемый символ: 1 (' ')


In [29]:
# Определим размер партии и размер буфера для ее перемещения. TensorFlow спроектирован, чтобы работать с бесконечными последовательностями, за всю сразу он не будет браться. 
# Вместо этого он оперирует элементами в специальном буфере.
BATCH_SIZE = 64
BUFFER_SIZE = 10000

# Переопределим датасет, параметризировав перетасовку партий с откидыванием десятичных знаков ее размера 
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

# Посмотрим, какими свойствами он теперь обладает. 64 характеризует размер партии, 100 – ее размер, причем тип жанных – 64-разрядное целое число со знаком.
dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [30]:
# Создадим переменную vocabulary size, которая характеризует размер словаря символов
vocab_size = len(vocab)

# Также нам понадобится так называемое встраивамое измерение
embedding_dim = 256

# И количество юнитов рекуррентной нейронной сети. Подождите, сейчас все встанет не свои места.
rnn_units = 1024

# Определим функцию – построитель модели, которая примет только что созданные аргументы, и размер партии
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  # Модель будет использовать функцию последовательности интерфейса Keras
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              # batch_input_shape определяет, какого размера должна быть входная партия и исключает создание векторов переменного размера. None здесь,
                              # чтобы охарактеризовать отсутствие временных шагов
                              batch_input_shape=[batch_size, None]),
    # Если возвратная нейросеть принимает на входе учебные данные и данные предыдущей итерации обучения, то Управляемый рекуррентный блок (GRU) использует фильтры сброса 
    # и обновления, тем самым очищая датасет от случайных вхождений. rnn_units описывает размерность выходного пространства, 
    tf.keras.layers.GRU(rnn_units,
                        # return_sequences – это булевый параметр, по умолчанию равный False. Он вернет часть выходной последовательности или всю ее целиком
                        return_sequences=True,
                        # Если True, последнее состояние для каждого образца с индексом i в партии будет использоваться в качестве исходного состояния для образца индекса i 
                        # в следующей партии.
                        stateful=True,
                        # Инициализируем матрицу весов, которая потребуется для гейтов сброса и обновления
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

# Построим модель и используем для этого вышеописанные параметры
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

# Проверим еще раз, чему равны величины партий, последовательностей и словарей. example_batch_predictions – понадобится для тестового предсказания последующих символов;
# Размеры для всех партий одинаковые
for input_example_batch, target_example_batch in dataset.take(2):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape)

(64, 100, 65)
(64, 100, 65)


In [31]:
# Последовательная модель описана, и вот ее краткие характеристики
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (64, None, 256)           16640     
_________________________________________________________________
gru_2 (GRU)                  (64, None, 1024)          3938304   
_________________________________________________________________
dense_2 (Dense)              (64, None, 65)            66625     
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


In [32]:
# Мы обучили нейросеть предсказывать символы, посмотрим, как это выглядит в зеркальном численном формате
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
# Метод squeeze удалит одномерность из тензора
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()
sampled_indices
# Здорово, мы на полпути к сочинению стихов!

array([26, 26, 14, 61,  6, 42, 47, 35, 34,  4, 48, 33, 44, 13, 57,  7, 52,
       39, 19, 36, 34,  1, 42, 52, 43, 48, 31, 56, 30, 61, 14, 62, 10, 33,
       36,  3, 30, 61, 60, 37, 10, 63, 59, 37, 34, 23, 11, 60, 52, 21,  1,
       25,  0, 39, 60, 25,  5, 51, 30, 21, 27, 22, 51, 53, 36, 50, 27, 27,
       27, 18, 12, 50, 14, 22, 12, 38, 49, 36,  1, 47, 21, 50, 61, 60, 26,
       59, 20, 54, 54, 33, 33, 41, 52, 30, 57,  3, 54, 52, 19,  6])

In [33]:
print("Ввод: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Следующий символ: \n", repr("".join(idx2char[sampled_indices ])))
# Это немного напоминает классический шифр сдвига на N символов, но уже на порядок сложнее, не так ли? Можно сказать, что мы создали свой шифр.

Ввод: 
 'My crown is in my heart, not on my head;\nNot decked with diamonds and Indian stones,\nNor to be seen:'

Следующий символ: 
 "NNBw,diWV&jUfAs-naGXV dnejSrRwBx:UX$RwvY:yuYVK;vnI M\navM'mRIOJmoXlOOOF?lBJ?ZkX iIlwvNuHppUUcnRs$pnG,"


In [34]:
# Перепишем функцию минимизации потерь, ярлыки и ненормализованные предсказания модели – logits
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

# Переопределим example_batch_loss и применим к объекту функцию loss
example_batch_loss = loss(target_example_batch, example_batch_predictions)
print("Величины партий, последовательностей и словарей: ", example_batch_predictions.shape)
# В большинстве обучающих сетей ошибка рассчитывается как разница между фактическим выходным значением и прогнозируемым выходным значением. Функция, используемая 
# для вычисления этой ошибки, известна как функция потерь. Ее еще называют функцией ошибки или затрат. Определим скалярные потери.
print("Скалярные потери: ", example_batch_loss.numpy().mean())

Величины партий, последовательностей и словарей:  (64, 100, 65)
Скалярные потери:  4.1733227


In [35]:
# Скомпилируем модель. Мы хотим минимизировать ошибку предсказания на имеющейся выборке букв, и потому используем метод оптимизации Adam.
model.compile(optimizer='adam', loss=loss)
# Зададим место сохранения промежуточных результатов
checkpoint_dir = './training_checkpoints'
# Префикс, первая часть названия чекпоинта, автоматически составится из названия выделенной папки и порядкового номера эпохи.
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

# Колбэк создаст чекпоинты прямо во время обучения нейросети. ModelCheckpoint, кстати, может создать Вам промежуточное состояние обучения, чтобы к нему было 
# удобнее вернуться позднее 
checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    # Веса здесь понимаются как сила воздействия перцептрона на конечное решение выбора буквы.
    save_weights_only=True)

# Снова разделим процесс на 10 эпох.
EPOCHS=10

# Объект history – это результат обучения РНС учебной частью данных в обозначенное количество эпох с созданием чекпоинтов
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

# Поскольку чекпоинты созданы, обозначим, что проводим обучение до последнего их них.
tf.train.latest_checkpoint(checkpoint_dir)
# Строим модель 
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size = 1)
# И используем самые точные веса последнего чекпоинта
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

# Спецификатор TensorShape характеризует форму тензора: мы знаем количество разрешений, но не знаем их размер, потому единица и None.
model.build(tf.TensorShape([1, None]))
model.summary()

Epoch 1/10
172/172 [==============================] - 874s 5s/step - loss: 2.6653
Epoch 2/10
172/172 [==============================] - 871s 5s/step - loss: 1.9619
Epoch 3/10
172/172 [==============================] - 906s 5s/step - loss: 1.6926
Epoch 4/10
172/172 [==============================] - 875s 5s/step - loss: 1.5442
Epoch 5/10
172/172 [==============================] - 872s 5s/step - loss: 1.4583
Epoch 6/10
172/172 [==============================] - 896s 5s/step - loss: 1.3974
Epoch 7/10
172/172 [==============================] - 888s 5s/step - loss: 1.3522
Epoch 8/10
172/172 [==============================] - 868s 5s/step - loss: 1.3144
Epoch 9/10
172/172 [==============================] - 914s 5s/step - loss: 1.2801
Epoch 10/10
172/172 [==============================] - 881s 5s/step - loss: 1.2470
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)  

In [48]:
# Определим функцию-генератор текста
def generate_text(model, start_string):
  # Определим, сколько символов сгенерировать
  num_generate = 1000

  # Векторизируем первую строку 
  input_eval = [char2idx[s] for s in start_string]
  # Добавим измерение на место с нулевым индексом с помощью метода expand_dims
  input_eval = tf.expand_dims(input_eval, 0)

  # Опустошим переменную, чтобы сохранить в нее результат
  text_generated = []

  # Температура – это так называемый гиперпараметр. Чем выше температура, тем менее предсказуем будет выходной текст.
  temperature = 1.0

  # Сбросим значения переменных модели
  model.reset_states()

  # Циклом for пробежимся
  for i in range(num_generate):
      predictions = model(input_eval)
      # Удалим измерения партии
      predictions = tf.squeeze(predictions, 0)

      # Опишем параметр предсказания
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # и передадим предсказанные символы следующей модели, также как и предыдущее скрытое состояние
      input_eval = tf.expand_dims([predicted_id], 0)

      # и присоединим каждое новое предсказание к предыдущим
      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))
  print(generate_text(model, start_string=u"Ромео: "))

model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

# И вот снова с нами Adam как средство улучшения качества предсказания
optimizer = tf.keras.optimizers.Adam()

# At sign tf.function – это декоратор TensorFlow, он преобразует функцию в граф. Допустим, мы инициализировали вектор V и добавили в него 6 элементов. Ребра E,
# описывают их взаимосвязи. Веса, которые нарастят элементы, будут как бы влиять на построение маршрута предсказания. GradientTape – это мощный интерфейс для вычисления градиентов. Нейронная сеть уточняет с каждой итерацией, как взаимосвязаны входные и выходные данные. Чем синее
# точка, в которой находится формула предсказания, тем точнее предсказывается результат. Внимательно соблюдаем отступы.
@tf.function
def train_step(inp, target):
    with tf.GradientTape() as tape:
        predictions = model(inp)
        loss = tf.reduce_mean(
            tf.keras.losses.sparse_categorical_crossentropy(
                target, predictions, from_logits=True))
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

    return loss


# Тренировочный шагов будет десять
EPOCHS = 10

for epoch in range(EPOCHS):
    start = time.time()

    # Инициализируем скрытое состояние в начале каждой эпохи, предварительно сбросив значения переменных модели.
    hidden = model.reset_states()

    # В цикле for отобразим промежуточные результаты тренировки
    for (batch_n, (inp, target)) in enumerate(dataset):
        loss = train_step(inp, target)

        if batch_n % 100 == 0:
            template = 'Эпоха {} Партия {} Потеря {}'
            print(template.format(epoch+1, batch_n, loss))

    # Настроим сохранение чекпоинтов каждые 5 секунд
    if (epoch + 1) % 5 == 0:
        model.save_weights(checkpoint_prefix.format(epoch=epoch))

    # Отобразим характеристики обучения, f здесь – это число с плавающей точкой
    print('Эпоха {} Партия {:.4f}'.format(epoch+1, loss))
    print('Время на одну эпоху {} сек\n'.format(time.time() - start))

model.save_weights(checkpoint_prefix.format(epoch=epoch))

Эпоха 1 Партия 0 Потеря 4.174645900726318
Эпоха 1 Партия 100 Потеря 2.3427305221557617
Эпоха 1 Партия 2.1591
Время на одну эпоху 939.772228717804 сек

Эпоха 2 Партия 0 Потеря 2.148317337036133
Эпоха 2 Партия 100 Потеря 1.8907116651535034
Эпоха 2 Партия 1.7703
Время на одну эпоху 925.9129273891449 сек

Эпоха 3 Партия 0 Потеря 1.812811255455017
Эпоха 3 Партия 100 Потеря 1.6637051105499268
Эпоха 3 Партия 1.6230
Время на одну эпоху 914.2517261505127 сек

Эпоха 4 Партия 0 Потеря 1.574855089187622
Эпоха 4 Партия 100 Потеря 1.5760409832000732
Эпоха 4 Партия 1.4947
Время на одну эпоху 922.9875419139862 сек

Эпоха 5 Партия 0 Потеря 1.420361042022705
Эпоха 5 Партия 100 Потеря 1.4625359773635864
Эпоха 5 Партия 1.4310
Время на одну эпоху 889.3739519119263 сек

Эпоха 6 Партия 0 Потеря 1.4059410095214844
Эпоха 6 Партия 100 Потеря 1.4020757675170898
Эпоха 6 Партия 1.4102
Время на одну эпоху 919.101603269577 сек

Эпоха 7 Партия 0 Потеря 1.3456593751907349
Эпоха 7 Партия 100 Потеря 1.331418752670288
Эп